In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import urllib.request
from bs4 import BeautifulSoup
import csv

import os
import re

import pickle

# Scrapping Epidemic Data

In [2]:
EPIDEMICS = [
    "MERS",
    "SARS",
    "SWINE",
    "MAD COW",
    "EBOLA",
    "INFLUENZA"
]

In [15]:
YEARS = range(2000, 2021)
WHO_BASE_URL = "https://www.who.int"

## Archive WHO articles

In [129]:
def page_soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, "html.parser")

def archive_article(url, year, text):
    safe_filename = url.replace("/", "_")
    
    if os.path.isfile(f'data/{year}/{safe_filename}.html'):
        return False
    
    with open(f'data/{year}/{safe_filename}.html', 'w') as file:
        file.write(str(text))
        
    return True

In [77]:
def archive_year_epidemic_dons(year):
    year_url = f"{WHO_BASE_URL}/csr/don/archive/year/{year}/en/"
    
    if not os.path.isdir(f'data/{year}'):
        os.mkdir(f'data/{year}')
    
    year_archive = page_soup(year_url).find("ul", {"class": "auto_archive"})
    
    for article_link in year_archive.findAll('a'):
        soup = page_soup(WHO_BASE_URL + article_link['href'])
        
        article_text = soup.find("div", {"id": "primary"})
        
        if not archive_article(article_link['href'], year, article_text):
            print(f'Article {article_link["href"]} archive already exists.')
            continue
        
    return True

In [78]:
for year in YEARS:
    print(f"Running for {year}.")
    archive_year_epidemic_dons(year)

Running for 2000.
Running for 2001.
Running for 2002.
Running for 2003.
Article /csr/don/2003_06_17a/en/ archive already exists.
Running for 2004.
Running for 2005.
Running for 2006.
Running for 2007.
Running for 2008.
Running for 2009.
Running for 2010.
Running for 2011.
Running for 2012.
Running for 2013.
Running for 2014.
Running for 2015.
Running for 2016.
Running for 2017.
Running for 2018.
Running for 2019.


## Analyze Archive

In [186]:
date_pattern = re.compile("([0-9]{4}_[0-9]+_[0-9]+|[0-9]+\-[a-zA-Z]+\-[0-9]{4})")

def parse_filename_date(filename):
    date_str = re.search(date_pattern, filename).group().replace("_", "/")
    
    return pd.to_datetime(date_str)

In [189]:
for year in YEARS:
    for filename in os.listdir(f'data/{year}'):
        with open(f'data/{year}/{filename}', "r") as file:
            article_date = parse_filename_date(filename)
            article_text = BeautifulSoup(file.read(), "html.parser")

## MERS

There is a lot of Excel data available on MERS outbreaks throughout the years. We should split each outbreak and analyze them seperately. This will be especially useful when reconciling separate excel files into one.

In [3]:
MERS_excels = None
with open("data/MERS_excels.pk", "rb") as file:
    MERS_excels = pickle.load(file)

### South Korea 2015

The MERS outbreak of 2015 in South Korea is the first set of data available. Upon initial inspection of the files, it becomes clear that these build on top of each other and therefore there's no need to aggregate all of them. Instead, we can just consider the last record available containing 188 patient records.

In [453]:
south_korea_url = MERS_excels[0]

In [454]:
south_korea_url

'https://www.who.int/csr/disease/coronavirus_infections/MERS-CoV-cases-rok-21Jul15.xlsx'

In [467]:
south_korea_df = pd.read_excel(south_korea_url, header=3)

#### Create Infection Date column

In [471]:
date_columns = [
    "Date of symptoms onset", 
    "Date of first hospitalization",
    "Date of laboratory confirmation",
    "Date of outcome"
]

In [472]:
for col in date_columns:
    south_korea_df[col] = pd.to_datetime(south_korea_df[col], errors='coerce')

In [473]:
south_korea_df["Date of infection"] = south_korea_df["Date of symptoms onset"]
# first fallback: first hospitalization
south_korea_df.loc[south_korea_df["Date of infection"].isna(), "Date of infection"] = south_korea_df.loc[south_korea_df["Date of infection"].isna(), "Date of first hospitalization"]
# second fallback: laboratory confirmation
south_korea_df.loc[south_korea_df["Date of infection"].isna(), "Date of infection"] = south_korea_df.loc[south_korea_df["Date of infection"].isna(), "Date of laboratory confirmation"]

In [476]:
irrelevant_columns = [
    "Case no.",
    "Age",
    "Sex",
    "Health care worker",
    "Comorbidities",
    "Date of symptoms onset", 
    "Date of first hospitalization",
    "Date of laboratory confirmation"
]

reduced_df = south_korea_df.drop(columns=irrelevant_columns)\
                           .rename({"Date of notification to WHO": "Last updated", "Status": "Outcome"}, axis=1)\
                           .reset_index(drop=True)

reduced_df.columns.name = ""
reduced_df = reduced_df[["Date of infection", "Outcome", "Date of outcome", "Last updated"]]

reduced_df.shape

(185, 4)

In [477]:
reduced_df.to_csv('data/MERS/MERS_south_korea_patients.csv')

#### Create Timeline

In [478]:
def build_timeline(df):
    infection_cumsum = df.groupby(df['Date of infection'].dt.date).size().cumsum()
    death_cumsum = df[df['Outcome'] == 'Deceased'].groupby(df['Date of outcome'].dt.date).size().cumsum()
    
    index_union = death_cumsum.index.union(infection_cumsum.index)
    
    return pd.DataFrame({
        "Infected": infection_cumsum, 
        "Dead": death_cumsum
    }, index=index_union).fillna(method='ffill').fillna(0).astype('int64')

In [479]:
timeline_df = build_timeline(reduced_df)

In [480]:
timeline_df.to_csv('data/MERS/MERS_south_korea_timeline.csv')

### Middle East 2017-2019

This is the other major outbreak happened in the Middle East from early 2017 on. All remaining records seem to involved this outbreak.

In [492]:
if os.path.isfile('data/MERS/MERS_middle_east_raw.csv'):
    middle_east_df = pd.read_csv('data/MERS/middle_east/raw.csv', index_col=0)
else:
    middle_east_df = pd.concat([pd.read_excel(url, header=None) for url in MERS_excels[9:]], sort=False).reset_index(drop=True)
    
    middle_east_df.to_csv('data/MERS/MERS_middle_east_raw.csv')
    
middle_east_df.shape

(6862, 17)

### Clean DataFrame

#### Drop Filler Rows

Most excel files contain many filler rows (for descriptions and explanations) that we need to ignore in our analysis. The threshold 2 was selected by iterative selection.

In [493]:
middle_east_df = middle_east_df.dropna(thresh=2)

middle_east_df.shape

(4472, 17)

#### Drop Duplicates

In [494]:
middle_east_df = middle_east_df.drop_duplicates()

In [495]:
middle_east_df.shape

(613, 17)

#### Remove header rows

Before performing any further action, we must keep track of column names so we use one of these rows as our default columns for the DataFrame.

In order to remove header rows, we must find a column with a consistent value type across all reports. Initially, we thought that the first column (usually named "Case no.") would solve our issue. As it turns out, there are some issues with this column so we move on to the next one (normally called "Date of notification to WHO (yyyy/mm/dd)"). Using this column removes a total of 5 rows from the DataFrame.

In [496]:
middle_east_df.columns = middle_east_df.iloc[0].tolist()

In [497]:
middle_east_df['Date of notification to WHO (yyyy/mm/dd)'] = pd.to_datetime(middle_east_df['Date of notification to WHO (yyyy/mm/dd)'], errors='coerce')

In [498]:
middle_east_df[middle_east_df['Date of notification to WHO (yyyy/mm/dd)'].isna()]

,Case no.,Date of notification to WHO (yyyy/mm/dd),Reporting country,City of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Exposure to MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),Last updated
4,Case no.,NaT,Reporting country,City of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Exposure to MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),Last updated
5027,Case no.,NaT,Reporting country,City of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Exposure to MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),NaN
5041,Case no.,NaT,Reporting country,Region of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Contact with a MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),Last updated
5056,Case no.,NaT,Reporting country,Region of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Contact with a MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),Last updated
5080,NaN,NaT,Age,Gender,National,Healthcare Worker,Region,City,Onset Date,Admission Date,Co-morbidity,Exposure to Dromedary,Symptomatic,Outcome,Date of Death,NaN,NaN
6767,Case no.,NaT,Reporting country,Region of residence,Age,Sex,Health care worker,Comorbidities,Exposure to camels,Camel milk consumption,Exposure to MERS-CoV cases,Date of symptoms onset (yyyy/mm/dd),Date of first hospitalization (yyyy/mm/dd),Date of laboratory confirmation (yyyy/mm/dd),Status,Date of outcome (yyyy/mm/dd),Last updated


In [499]:
middle_east_df = middle_east_df.dropna(subset=['Date of notification to WHO (yyyy/mm/dd)'])

middle_east_df.shape

(607, 17)

There seems to be one report that was formatted differently than all other ones (as can be seen above in row #332). For this case, we will have to manually assign the values to the correct columns using the column names shown above and hope that it fixes it.

In [500]:
misformatted_row_indexes = middle_east_df[middle_east_df['Reporting country'].str.isnumeric()].index

In [501]:
misformatted_df = middle_east_df.loc[misformatted_row_indexes].copy()

In [502]:
misformatted_df['Reporting country'] = 'Saudi Arabia'
misformatted_df['City of residence'] = middle_east_df.loc[misformatted_row_indexes]['Comorbidities']
misformatted_df['Age'] = middle_east_df.loc[misformatted_row_indexes]['Reporting country']
misformatted_df['Sex'] = middle_east_df.loc[misformatted_row_indexes]['City of residence']
misformatted_df['Health care worker'] = middle_east_df.loc[misformatted_row_indexes]['Sex']
misformatted_df['Comorbidities'] = middle_east_df.loc[misformatted_row_indexes]['Exposure to MERS-CoV cases']
misformatted_df['Exposure to camels'] = middle_east_df.loc[misformatted_row_indexes]['Date of symptoms onset (yyyy/mm/dd)']
misformatted_df['Status'] = middle_east_df.loc[misformatted_row_indexes]['Date of laboratory confirmation (yyyy/mm/dd)']
misformatted_df['Date of symptoms onset (yyyy/mm/dd)'] = middle_east_df.loc[misformatted_row_indexes]['Exposure to camels']
misformatted_df['Date of outcome (yyyy/mm/dd)'] = middle_east_df.loc[misformatted_row_indexes]['Status']
misformatted_df['Date of first hospitalization (yyyy/mm/dd)'] = middle_east_df.loc[misformatted_row_indexes]['Camel milk consumption']
misformatted_df['Date of laboratory confirmation (yyyy/mm/dd)'] = np.nan

In [503]:
middle_east_df.loc[misformatted_row_indexes, :] = misformatted_df

#### Reformat Identifier Columns

In [379]:
middle_east_df.replace({
    "male": "M",
    "female": "F",
    "Alive ": "Alive",
    "Survivied ": "Survived",
    "Recovered": "Survived",
    "Died": "Deceased"
}, inplace=True)

#### Sort DataFrame chronologically

Firstly, we need to handle the cases of empty Last updated value. We will consider the date of notification to be this last update.

In [380]:
middle_east_df.loc[:, "Last updated"] = pd.to_datetime(middle_east_df["Last updated"], errors='coerce')

In [381]:
nan_last_updated_mask = middle_east_df[middle_east_df["Last updated"].isna()].index

In [382]:
middle_east_df.loc[nan_last_updated_mask, "Last updated"] = middle_east_df.loc[nan_last_updated_mask, "Date of notification to WHO (yyyy/mm/dd)"]

In [383]:
middle_east_df = middle_east_df.sort_values(by='Last updated').reset_index(drop=True)

#### Remove duplicate patients

First, we remove duplicated patients by their quasi-identifiers (defined below) including their `"Status"`. This allows us to remove those duplicates that occur due to an update (with no changes).

In [384]:
quasi_identifiers = [
    "Date of notification to WHO (yyyy/mm/dd)", 
    "Reporting country", 
    "City of residence", 
    "Age", 
    "Sex", 
    "Date of symptoms onset (yyyy/mm/dd)", 
    "Date of first hospitalization (yyyy/mm/dd)"
]

In [385]:
middle_east_df = middle_east_df.drop_duplicates(keep='last', subset=(quasi_identifiers + ["Status"]))

middle_east_df.shape

(574, 17)

Next, we look at duplicated patients given their `quasi_identifiers` and keep the most up-to-date record for each patient.

In [386]:
middle_east_df = middle_east_df.drop_duplicates(keep='last', subset=quasi_identifiers)

middle_east_df.shape

(546, 17)

We will now assumed that all other patients whose `"Status"` is `Alive` have gone on to survive the disease. Therefore, we can relabel them as `Survived`

In [387]:
middle_east_df.replace({"Alive": "Survived"}, inplace=True)

#### Drop unconfirmed cases

There are 48 patients for which the laboratory confirmation date is empty. This number is non-negligeable for the scale of data available, but we should still drop these as we can't be sure that they represent people who were actually infected.

In [388]:
middle_east_df = middle_east_df[~middle_east_df["Date of laboratory confirmation (yyyy/mm/dd)"].isna()]

In [389]:
middle_east_df.shape

(499, 17)

#### Create Infection Date column

First, we want to convert all dates into datetime types and drop any unconfirmed cases (there are only 48 of them).

In [390]:
date_columns = [
    "Date of symptoms onset (yyyy/mm/dd)", 
    "Date of first hospitalization (yyyy/mm/dd)",
    "Date of laboratory confirmation (yyyy/mm/dd)",
    "Date of outcome (yyyy/mm/dd)"
]

In [391]:
for col in date_columns:
    middle_east_df[col] = pd.to_datetime(middle_east_df[col], errors='coerce')

Next, we want to define a new column that will be used to simplify and unify the three different infection related date columns. We will prioritize the symptoms column as our main infection start column (even when there is no laboratory confirmation). For null values, we will fallback to the first hospitalization and then laboratory confirmation.

In [392]:
middle_east_df["Date of infection"] = middle_east_df["Date of symptoms onset (yyyy/mm/dd)"]
# first fallback: first hospitalization
middle_east_df.loc[middle_east_df["Date of infection"].isna(), "Date of infection"] = middle_east_df.loc[middle_east_df["Date of infection"].isna(), "Date of first hospitalization (yyyy/mm/dd)"]
# second fallback: laboratory confirmation
middle_east_df.loc[middle_east_df["Date of infection"].isna(), "Date of infection"] = middle_east_df.loc[middle_east_df["Date of infection"].isna(), "Date of laboratory confirmation (yyyy/mm/dd)"]

#### Drop irrelevant columns

In [407]:
irrelevant_columns = [
    "Case no.",
    "Date of notification to WHO (yyyy/mm/dd)",
    "Reporting country",
    "City of residence",
    "Age",
    "Sex",
    "Health care worker",
    "Comorbidities",
    "Exposure to camels",
    "Camel milk consumption",
    "Exposure to MERS-CoV cases",
    "Date of symptoms onset (yyyy/mm/dd)", 
    "Date of first hospitalization (yyyy/mm/dd)",
    "Date of laboratory confirmation (yyyy/mm/dd)",
]

reduced_df = middle_east_df.drop(columns=irrelevant_columns)\
                           .rename({"Date of outcome (yyyy/mm/dd)": "Date of outcome", "Status": "Outcome"}, axis=1)\
                           .reset_index(drop=True)

reduced_df.columns.name = ""
reduced_df = reduced_df[["Date of infection", "Outcome", "Date of outcome", "Last updated"]]

reduced_df.shape

(499, 4)

In [431]:
reduced_df.to_csv('data/MERS/MERS_middle_east_patients.csv')

#### Create Timeline

In [450]:
def build_timeline(df):
    infection_cumsum = df.groupby(df['Date of infection'].dt.date).size().cumsum()
    death_cumsum = df[df['Outcome'] == 'Deceased'].groupby(df['Date of outcome'].dt.date).size().cumsum()
    
    index_union = death_cumsum.index.union(infection_cumsum.index)
    
    return pd.DataFrame({
        "Infected": infection_cumsum, 
        "Dead": death_cumsum
    }, index=index_union).fillna(method='ffill').fillna(0).astype('int64')

In [451]:
timeline_df = build_timeline(reduced_df)

In [446]:
timeline_df.to_csv('data/MERS/MERS_middle_east_timeline.csv')

## Ebola

In [532]:
ebola_df = pd.read_csv('data/ebola_data_db_format.csv')

In [540]:
confirmed_df = ebola_df[ebola_df["Indicator"].isin(["Cumulative number of confirmed Ebola cases", "Cumulative number of confirmed Ebola deaths"])]

In [545]:
reduced_df = confirmed_df.groupby(['Country', 'Date', 'Indicator'])['value'].sum().unstack('Indicator').fillna(method='ffill').astype('int64')

In [538]:
confirmed_df[~(confirmed_df["Cumulative number of confirmed Ebola cases"] >= confirmed_df["Cumulative number of confirmed Ebola deaths"])]

Indicator           Cumulative number of confirmed Ebola cases  \
Country Date                                                     
Liberia 2014-10-03                                         931   
        2014-10-08                                         941   
        2014-10-10                                         943   
        2014-10-15                                         950   
        2014-10-22                                         965   
        2014-10-25                                         965   
        2015-02-10                                        3146   
        2015-02-11                                        3146   
        2015-02-12                                        3146   
        2015-02-13                                        3147   
        2015-02-16                                        3149   
        2015-02-17                                        3149   
        2015-02-18                                        3149   
        2015-02-19                                        3152   
        2015-02-20                                        3152   
        2015-02-23                                        3153   
        2015-02-25                                        3153   
        2015-02-26                                        3153   
        2015-02-27                                        3153   
        2015-03-02                                        3153   
        2015-03-04                                        3153   
        2015-03-05                                        3150   
        2015-03-06                                        3150   
        2015-03-10                                        3150   
        2015-03-11                                        3150   
        2015-03-12                                        3150   
        2015-03-13                                        3150   
        2015-03-16                                        3150   
        2015-03-17                                        3150   
        2015-03-18                                        3150   
...                                                        ...   
        2015-10-22                                        3151   
        2015-10-23                                        3151   
        2015-10-26                                        3151   
        2015-10-27                                        3151   
        2015-10-29                                        3151   
        2015-10-30                                        3151   
        2015-11-03                                        3151   
        2015-11-05                                        3151   
        2015-11-06                                        3151   
        2015-11-10                                        3151   
        2015-11-11                                        3151   
        2015-11-12                                        3151   
        2015-11-16                                        3151   
        2015-11-17                                        3151   
        2015-11-20                                        3151   
        2015-11-24                                        3151   
        2015-11-26                                        3151   
        2015-12-01                                        3151   
        2015-12-02                                        3151   
        2015-12-03                                        3151   
        2015-12-09                                        3151   
        2015-12-10                                        3151   
        2015-12-11                                        3151   
        2015-12-15                                        3151   
        2015-12-16                                        3151   
        2015-12-17                                        3151   
        2015-12-22                                        3151   
        2015-12-23                      

# Archive

In [66]:
dates = pd.to_datetime(drop_rows(oman_df)['Date of notification to WHO (yyyy/mm/dd)'])

In [88]:
date_range = pd.date_range(dates.min(), dates.max(), freq='D')

In [96]:
oman_df_date_distribution = dates.groupby(dates.dt.date, as_index=True).count().reindex(date_range, fill_value=0)

In [103]:
oman_df_date_distribution["2017-01-01":"2017-12-31"].groupby(oman_df_date_distribution["2017-01-01":"2017-12-31"].index.month).count()

1     30
2     28
3     31
4     30
5     31
6     30
7     31
8     31
9     30
10    31
11    30
12    31
Name: Date of notification to WHO (yyyy/mm/dd), dtype: int64